In [42]:
import requests
import time
import os
import numpy as np
import subprocess
from operator import itemgetter
import glob
import random
import fnmatch
import itertools
from bs4 import BeautifulSoup
from urllib.request import urlopen

repo_pathname = '/media/smcl/Audio/epc_pages/'

os.chdir(repo_pathname)

!ls

EPC_Link_Check_Script.ipynb   LICENSE
EPC_Link_Cleaning.ipynb       README.md
EPC_Link_Status_Table.csv     writing.upenn.edu_08_06_2018_151036
Global_Replacement_Table.csv


In [43]:
import csv

urls_lol = []

row_set = set()

with open('EPC_Link_Status_Table.csv') as fi:
    csv_reader = csv.reader(fi)
    for row in csv_reader:
        if '|'.join(row[:2]) not in row_set:   ### Removing duplicates
            urls_lol.append(row)
            row_set.add('|'.join(row[:2]))

header = urls_lol[0]
urls_lol = urls_lol[1:]        

len(urls_lol)

4604

In [44]:
import json

def get_wayback_url(url):
    wayback_url = ''
    api_url = 'https://archive.org/wayback/available?url=' + url
    try:
        page = urlopen(api_url).read().decode('utf8')
    except Exception as e:
        print(e)
        print(api_url)
    json_object = json.loads(page)
    try:
        wayback_url = json_object['archived_snapshots']['closest']['url']
    except:
        try:
            api_url = 'https://archive.org/wayback/available?url=' + json_object['url']
            page2 = urlopen(api_url).read().decode('utf8')
            json_object = json.loads(page2)
            wayback_url = json_object['archived_snapshots']['closest']['url']
        except Exception as e:
            #print(e)
            #print(url)    
            pass
    return wayback_url

In [45]:
get_wayback_url('https://www.poetryfoundation.org/poem/239052')

'http://web.archive.org/web/20160311065022/http://www.poetryfoundation.org/poem/239052'

In [46]:
print(header)

random.choice(urls_lol)

['url', 'linked from', 'server response', 'redirect link', 'replacement link']


['http://pmc.iath.virginia.edu/issue.905/16.1emerson.html',
 '/epc/authors/glazier/docs/glazier-crit-cites.html',
 '200',
 '',
 '']

In [47]:
get_wayback_url('https://www.bookwire.com/ErrorPage?is404=1')

''

In [48]:
redirect_blacklist = ('http://writing.upenn.edu/wh/people/fellows', 'https://www.nyq.org/', 'https://www.poets.org/', 'https://www.bookwire.com/ErrorPage?is404=1', 'http://commutiny.net')

In [49]:
## Making updates within table

import requests

urls_lol_2 = []

counter = 0

for row in urls_lol:
    if counter%200==0:
        print(counter)
    counter+=1
    temp_row = row
    if (temp_row[3]!='')&(temp_row[4]==''):
        try:
            url = temp_row[3]
            url = url.replace('poets.org/poets/poets.cfm?prmID=', 'poets.org/poet.php/prmPID/')
            r = requests.head(url)
            if r.status_code in (301, 302, 303, 304):
                new_url = r.headers['Location'] ######## <-------------------
                if new_url[0] == '/':
                    base_url = 'https://' + url.split('/')[2]
                    new_url = os.path.join(base_url, new_url)
                elif 'http' not in new_url:
                    base_url = 'https://' + '/'.join(url.split('/')[2:-1])
                    new_url = os.path.join(base_url, new_url)
                if (new_url not in redirect_blacklist) & ('http' in new_url):
                    if ('ucpress.edu' in url.lower()) or ('poetryfoundation.org' in url.lower()):
                        new_url = url
                    if new_url!='':
                        temp_row[3] = new_url
                else:
                    if new_url not in ('https://www.press.uchicago.edu/'):
                        new_url = get_wayback_url(url)
                    if new_url!='':
                        temp_row[4] = new_url
                        print("redirecting again: ")
                        print(url)
                        print(new_url)
                        print('------')
            elif r.status_code in (403, 404):
                new_url = get_wayback_url(url)
                if ('ucpress.edu' in url.lower()) or ('poetryfoundation.org' in url.lower()):
                    temp_row[4] = url
                elif new_url!='':
                    temp_row[4] = new_url
                    print('dead redirect: ')
                    print(url)
                    print(new_url)
        except Exception as e:
            if 'people.well.com' not in url.lower():
                print('****')
                print(e)
                print(url)
                new_url = get_wayback_url(url)
                temp_row[4] = new_url
                print(new_url)
                print('*****')
    urls_lol_2.append(temp_row)

0
200
dead redirect: 
https://www.nypl.org/locations/tid/36/exhibitions?id=430
http://web.archive.org/web/20140531084712/http://www.nypl.org:80/locations/tid/36/exhibitions?id=430
****
('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
http://ww17.comparisonics.com/ColorWaveformDemo.mov

*****
****
('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
http://ww17.comparisonics.com/gallery.html

*****
redirecting again: 
https://forward.com/articles/7344/
http://web.archive.org/web/20110401203747/http://www.forward.com/articles/7344/
------
400
dead redirect: 
https://cla.umn.edu/joglars/xe/xexoxial/xexoxial.html
http://web.archive.org/web/20061021185242/http://cla.umn.edu:80/joglars/xe/xexoxial/xexoxial.html
****
Invalid URL '/Book/index.cfm?GCOI=93780100173080': No schema supplied. Perhaps you meant http:///Book/index.cfm?GCOI=93780100173080?
/Book/index.cfm?GCOI=93780100173080

*****
600
dead redirect: 
https://brocku.ca/precipice/


*****
****
Invalid URL '/Details/28131/Poems-from-the-Maghreb-Introduction-and-Selections': No schema supplied. Perhaps you meant http:///Details/28131/Poems-from-the-Maghreb-Introduction-and-Selections?
/Details/28131/Poems-from-the-Maghreb-Introduction-and-Selections

*****
1800
****
Invalid URL '/LYnaZ/salon3.html': No schema supplied. Perhaps you meant http:///LYnaZ/salon3.html?
/LYnaZ/salon3.html

*****
****
Invalid URL '/Book/index.cfm?GCOI=93780100623700': No schema supplied. Perhaps you meant http:///Book/index.cfm?GCOI=93780100623700?
/Book/index.cfm?GCOI=93780100623700

*****
****
Invalid URL '/Book/index.cfm?GCOI=93780100618720': No schema supplied. Perhaps you meant http:///Book/index.cfm?GCOI=93780100618720?
/Book/index.cfm?GCOI=93780100618720

*****
****
Invalid URL '/Book/index.cfm?GCOI=93780100618720': No schema supplied. Perhaps you meant http:///Book/index.cfm?GCOI=93780100618720?
/Book/index.cfm?GCOI=93780100618720

*****
2000
2200
dead redirect: 
https://bunchecent


*****
****
Invalid URL '/KNOaZ/': No schema supplied. Perhaps you meant http:///KNOaZ/?
/KNOaZ/

*****
4000
****
Invalid URL '/PnSUZ/': No schema supplied. Perhaps you meant http:///PnSUZ/?
/PnSUZ/

*****
****
Invalid URL '/XgpgZ/': No schema supplied. Perhaps you meant http:///XgpgZ/?
/XgpgZ/

*****
4200
****
Invalid URL '/Book/index.cfm?GCOI=93780100808520': No schema supplied. Perhaps you meant http:///Book/index.cfm?GCOI=93780100808520?
/Book/index.cfm?GCOI=93780100808520

*****
****
Invalid URL '/PZOTZ/Book/index.cfm?GCOI=93780100172350': No schema supplied. Perhaps you meant http:///PZOTZ/Book/index.cfm?GCOI=93780100172350?
/PZOTZ/Book/index.cfm?GCOI=93780100172350

*****
dead redirect: 
http://www.writing.upenn.edu/~wh/combo/
http://web.archive.org/web/20061024041358/http://www.writing.upenn.edu:80/~wh/combo/
****
Invalid URL '/Book/index.cfm?GCOI=93780100631100': No schema supplied. Perhaps you meant http:///Book/index.cfm?GCOI=93780100631100?
/Book/index.cfm?GCOI=937801006311

In [54]:
random.sample([item for item in urls_lol_2 if str(item[2])=='301'], 10)

[['http://www.writing.upenn.edu/%7Ewhfellow/rich.html',
  '/epc/authors/creeley/obit/rich.html',
  '301',
  'http://www.writing.upenn.edu/~whfellow/rich.html',
  'http://web.archive.org/web/20080530230518/http://www.writing.upenn.edu:80/%7Ewhfellow/rich.html'],
 ['http://www.writing.upenn.edu/pennsound/x/Weiner.html',
  '/epc/authors/weiner/cj/index.html',
  '301',
  'http://writing.upenn.edu/pennsound/x/Weiner.php',
  ''],
 ['http://www.highlandsnews.com/articles/2008/03/20/galax_news/03poet_dies.txt',
  '/epc/authors/williams_jonathan/index.html',
  '301',
  'https://www.highlandsnews.com/articles/2008/03/20/galax_news/03poet_dies.txt',
  'http://web.archive.org/web/20090816184641/http://www.highlandsnews.com/articles/2008/03/20/galax_news/03poet_dies.txt'],
 ['http://www.english.uiuc.edu/maps/poets/s_z/zukofsky/zukofsky.htm',
  '/epc/authors/zukofsky/zuk.pub.html',
  '301',
  'http://www.english.illinois.edu/maps/poets/s_z/zukofsky/zukofsky.htm',
  ''],
 ['http://www.poetryfoundatio

In [55]:
import csv

header_2 = header

with open('EPC_Link_Status_Table.csv', 'w') as fo:
    csv_writer = csv.writer(fo)
    csv_writer.writerow(header_2)
    csv_writer.writerows(urls_lol_2)

In [ ]:
import csv

urls_404 = []

for row in urls_lol:
    if row[2] in ('404', 'dead'):
        urls_404.append(row[0])

urls_404 = sorted(urls_404)
            
len(urls_404)

In [ ]:
import csv

urls_403 = []

for row in urls_lol:
    if row[2] in ('403'):
        urls_403.append(row[0])

urls_403 = sorted(urls_403)
            
len(urls_403)

In [ ]:
from urllib.request import urlopen
import json
import random

urls = urls_403

random.shuffle(urls)

url_dict = {}

for url in urls:
    api_url = 'https://archive.org/wayback/available?url=' + url
    try:
        page = urlopen(api_url).read().decode('utf8')
    except Exception as e:
        print(e)
        print(api_url)
    json_object = json.loads(page)
    try:
        wayback_url = json_object['archived_snapshots']['closest']['url']
        print(wayback_url)
        url_dict[url] = wayback_url
    except:
        try:
            api_url = 'https://archive.org/wayback/available?url=' + json_object['url']
            page2 = urlopen(api_url).read().decode('utf8')
            json_object = json.loads(page2)
            wayback_url = json_object['archived_snapshots']['closest']['url']
            print(wayback_url)
            url_dict[url] = wayback_url
        except:
            print(page)

In [ ]:

urls_301 = []

for row in urls_lol:
    if row[3]!='':
        urls_301.append(row[3])

urls_301 = sorted(urls_301)
            
len(urls_301)

In [ ]:
url = 'https://google.com/goodie/index.html'



base_url = 'https://' + '/'.join(url.split('/')[2:-1])

base_url

In [ ]:
for pair in url_301_dict.items():
    print(pair)

In [ ]:
import requests

url_301_dict = {}

for url in urls_301:
    try:
        r = requests.head(url)
        if r.status_code in (301, 302, 303, 304):
            url_301_dict[url] = r.headers['Location'] ######## <-------------------
    except Exception as e:
        print(e)

In [ ]:
for key in url_301_dict:
    if url_301_dict[key] != key:
        print(key)
        print(url_301_dict[key])
        print('---')

In [ ]:
#header

header_2 = ['url', 'linked from', 'server response', 'redirect link', 'replacement link']

header_2

In [ ]:
urls_lol_2 = []

for row in urls_lol:
    temp_row = row
    if row[3]!='':
        try: temp_row[3] = url_301_dict[row[3]]
        except: pass
    if row[2] in ('404', 'dead'):
        try: temp_row[4] = url_dict[row[0]]
        except: pass
    urls_lol_2.append(temp_row)

In [ ]:
urls_lol_2 = []

for row in urls_lol:
    temp_row = row
    if row[3]!='':
        try: temp_row[3] = url_301_dict[row[3]]
        except: pass
    if row[2] in ('403'):
        try: temp_row[4] = url_dict[row[0]]
        except: pass
    urls_lol_2.append(temp_row)
    print(temp_row)

In [ ]:
random.sample(urls_lol_2, 10)

In [ ]:
!wc -l 'EPC_Link_Status_Table.csv'